In [1]:
!git clone https://github.com/WebCrawlerr/GAT.git

Cloning into 'GAT'...
remote: Enumerating objects: 142, done.
remote: Counting objects: 100% (142/142), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 142 (delta 76), reused 94 (delta 42), pack-reused 0 (from 0)
Receiving objects: 100% (142/142), 116.66 KiB | 5.83 MiB/s, done.
Resolving deltas: 100% (76/76), done.


In [2]:
# Pobranie zmian
!git pull origin filter-belka

fatal: not a git repository (or any parent up to mount point /kaggle)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


In [3]:
# Wyświetla listę plików w pobranym folderze
!ls -R  GAT

GAT:
main.py  prepare_leash_data.py	run_experiments.py  tests
models	 requirements.txt	src

GAT/models:
best_model.pth	fold_0_best_model.pth

GAT/src:
config.py	    dataset.py	 model.py     __pycache__  utils.py
data_processing.py  features.py  optimize.py  train.py

GAT/src/__pycache__:
config.cpython-312.pyc		 model.cpython-312.pyc
data_processing.cpython-312.pyc  optimize.cpython-312.pyc
dataset.cpython-312.pyc		 train.cpython-312.pyc
features.cpython-312.pyc	 utils.cpython-312.pyc

GAT/tests:
test_cv.py	  test_optimize.py  test_plotting.py   verify_plots.py
test_features.py  test_pipeline.py  verify_metrics.py


In [4]:
import shutil
import os
import sys
# 1. Zmiana katalogu roboczego (odpowiednik komendy 'cd')
os.chdir('./GAT')
print(f"Aktualny katalog: {os.getcwd()}")

Aktualny katalog: /kaggle/working/GAT


In [5]:
# Najpierw sprawdźmy wersję po restarcie (czasem GPU ma starszego PyTorcha)
import torch
version = torch.__version__
print(f"PyTorch version: {version}")

# Instalacja PyG i bibliotek pomocniczych (korzystamy z gotowych paczek .whl, żeby nie kompilować przez 20 min)
# UWAGA: Ta komenda dynamicznie dobiera link do wersji Twojego PyTorcha
torch_version_suffix = version.split('+')[0].replace('.', '') # np. 260 lub 240
cuda_version_suffix = 'cu' + torch.version.cuda.replace('.', '') # np. cu121

print(f"Instaluję dla: torch-{torch_version_suffix} + {cuda_version_suffix}")

# Instalacja zależności
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv \
  -f https://data.pyg.org/whl/torch-{version}.html

# Instalacja reszty z Twojego pliku (bez ponownego instalowania torch-geometric, bo zrobiliśmy to wyżej)
!pip install -r requirements.txt

PyTorch version: 2.6.0+cu124
Instaluję dla: torch-260 + cu124
Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 118.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 92.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 94.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:0

In [6]:
import torch
import torch_geometric

print(f"CUDA dostępne: {torch.cuda.is_available()}")
print(f"Wersja PyTorch: {torch.__version__}")
print(f"Wersja PyG: {torch_geometric.__version__}")

# Próba importu kluczowych warstw
try:
    from torch_geometric.nn import GATConv
    print("✅ GATConv zaimportowany pomyślnie!")
except ImportError as e:
    print(f"❌ Błąd importu GATConv: {e}")

# Sprawdzenie czy mamy backendy (scatter/sparse)
# PyG 2.7+ potrafi czasem działać bez nich (wolniej), ale warto sprawdzić
try:
    import torch_scatter
    print("✅ torch_scatter obecny")
except ImportError:
    print("⚠️ torch_scatter BRAK (Może to spowolnić trening lub wywołać błąd)")

try:
    import torch_sparse
    print("✅ torch_sparse obecny")
except ImportError:
    print("⚠️ torch_sparse BRAK (Wymagany dla niektórych operatorów)")

CUDA dostępne: True
Wersja PyTorch: 2.6.0+cu124
Wersja PyG: 2.7.0
✅ GATConv zaimportowany pomyślnie!
✅ torch_scatter obecny
✅ torch_sparse obecny


In [7]:
#!python main.py --raw_file /kaggle/input/bindingdb-smiles/BindingDB_All.tsv --processed_dir ./data/processed --cv 5

In [8]:
#!python main.py --optimize --n_trials 15 --raw_file /kaggle/input/bindingdb-smiles/BindingDB_All.tsv --processed_dir ./data/processed
#--cv 5

In [9]:
#!python run_experiments.py --processed_dir ./data/processed --raw_file /kaggle/input/bindingdb-smiles/BindingDB_All.tsv

In [10]:
#PRZYGOTOWANIE DANYCH BELKA
!python prepare_leash_data.py --input_file /kaggle/input/leash-BELKA/train.csv --output_dir /kaggle/working/ --protein BRD4 --negative_rate 1

Starting processing...
Input File: /kaggle/input/leash-BELKA/train.csv
Output Directory: /kaggle/working/
Target Protein: BRD4
Negative Sample Rate: 1.0
Processing chunks: 296it [22:29,  4.56s/it]

Processing Complete!
Total rows processed: 295246830
Rows saved: 98415610
Positives saved: 456964
Negatives saved: 97958646
Output saved to: /kaggle/working/leash_brd4_filtered.csv


In [11]:
import os
import shutil
from IPython.display import FileLink

# Nazwa pliku, którego szukamy
zip_name = 'GAT_BRD4_Results.zip'

# 1. Znajdź plik w całym systemie plików roboczych
found_path = None
for root, dirs, files in os.walk('/kaggle/working'):
    if zip_name in files:
        found_path = os.path.join(root, zip_name)
        break

if found_path:
    print(f"✅ Znaleziono plik tutaj: {found_path}")
    
    # 2. Przenieś go do głównego katalogu (Dla pewności)
    target_path = f'/kaggle/working/{zip_name}'
    
    if found_path != target_path:
        shutil.move(found_path, target_path)
        print(f"📦 Przeniesiono plik do głównego katalogu: {target_path}")
    
    # 3. Zmień katalog roboczy na główny (żeby FileLink zadziałał)
    os.chdir('/kaggle/working')
    
    # 4. Wyświetl link
    print("👇 Kliknij poniżej:")
    display(FileLink(zip_name))
else:
    print("❌ Nie znaleziono pliku ZIP. Upewnij się, że poprzednia komórka wykonała się bez błędów.")
    # Sprawdźmy co w ogóle mamy na dysku
    print("Lista plików w obecnym folderze:")
    print(os.listdir('.'))

❌ Nie znaleziono pliku ZIP. Upewnij się, że poprzednia komórka wykonała się bez błędów.
Lista plików w obecnym folderze:
['requirements.txt', 'main.py', 'run_experiments.py', 'tests', 'models', '.git', '.gitignore', 'src', 'prepare_leash_data.py']
